In [1]:
from tqdm.notebook import tqdm  as tqdm
import ipyleaflet as ipy 
import ipywidgets as ipyw
from ipyleaflet import GeoJSON, Map, Marker
from shapely.geometry import shape
import json
import requests

from datetime import datetime

from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)
import pandas as pd
import itables.options as opt
opt.columnDefs = [{"className":"dt-left", "targets": "_all"}]
opt.lengthMenu = [50, 100, 200, 500]
import urllib, json, requests
import ipywidgets as widgets


<IPython.core.display.Javascript object>

In [2]:
TIME_API = "http://time-api.loci.cat/api/v1"

In [3]:
#visualise LGA 

def get_geom(loci_uri):
    payload = {
        "uri": loci_uri
    }
    url = TIME_API  + "/location/geometry"
    r = requests.get(url, params=payload)
    res = r.json()
    #get the first geom result
    return res

# Loc-I Change Over Time Demo

This notebook demonstrates the prototype Loc-I change over time API available at http://time-api.loci.cat

Some basic API functionality is introduced and a selection of use cases will be demonstrated below.

## Loc-I Time API Basics

### Search for a feature by name/label/code

Example: Find features with label of "Fraser"

In [4]:
w = widgets.Text(
    value='Maribyrnong',
    placeholder='Maribyrnong',
    description='Input label:',
    disabled=False
)


In [5]:
display(w)

Text(value='Maribyrnong', description='Input label:', placeholder='Maribyrnong')

In [6]:
API_ENDPOINT = "/location/find-by-label"

print("Issuing query to /location/find-by-label API endpoint. One moment please...")
url = TIME_API + API_ENDPOINT
payload = {
    'query': w.value
}
r = requests.get(url, params=payload)
jsondata = r.json()
#jsondata
df = pd.DataFrame (jsondata, columns = ['uri', 'match'])
df

Issuing query to /location/find-by-label API endpoint. One moment please...


uri,match


### Get a Loc-I Feature at a point in time

Example: Find the CED Commonwealth Electoral District (CED) of Maribyrnong at a point in time, 7th September 2017.

In [7]:
SOURCE_CED = "https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/maribyrnong"
TIME_VARIABLE = "2017-09-07"

API_ENDPOINT = "/temporal/feature"

print("Issuing query to /temporal/feature API endpoint with source CED " + SOURCE_CED + " and time at " + TIME_VARIABLE + ". One moment please...")

url = TIME_API + API_ENDPOINT
payload = {
    'uri': SOURCE_CED,
    'time' : TIME_VARIABLE
}
r = requests.get(url, params=payload)
jsondata = r.json()
print(json.dumps(jsondata, indent=4))

mydict = {k: [v] for k, v in jsondata.items()}
df = pd.DataFrame (mydict, columns = mydict.keys())
df

Issuing query to /temporal/feature API endpoint with source CED https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/maribyrnong and time at 2017-09-07. One moment please...
{
    "uri": "https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/maribyrnong/2017-07",
    "feature_type": "https://linked.data.gov.au/def/asgs#CommonwealthElectoralDivision",
    "valid_from": "2017-07-01",
    "valid_to": "2018-06-30",
    "code": "228",
    "name": "Maribyrnong",
    "ced_code16": "228",
    "ced_name16": "Maribyrnong",
    "areasqkm16": "72.514600000000002"
}


uri,feature_type,valid_from,valid_to,code,name,ced_code16,ced_name16,areasqkm16


Show the geometry for the feature on a map...

In [8]:
#draw the geometry for the feature on a map
SOURCE_CED_URI=jsondata['uri']
source_geojson_data = get_geom(SOURCE_CED_URI)
g= shape(source_geojson_data)

x_coord = g.centroid.coords[0][1]
y_coord = g.centroid.coords[0][0]

map = ipy.Map(center=[x_coord, y_coord], zoom=11)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

geo_json1 = GeoJSON(data=source_geojson_data, style = {'color': 'red', 'opacity':1, 'weight':1.9, 'fillOpacity':0.5})
map.add_layer(geo_json1)
map

Map(center=[-37.7533330724475, 144.86146427671474], controls=(ZoomControl(options=['position', 'zoom_in_text',…

Show the same CED for 2021-05-06

In [9]:
SOURCE_CED = "https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/maribyrnong"
TIME_VARIABLE = "2021-05-06"
API_ENDPOINT = "/temporal/feature"

print("Issuing query to /temporal/feature API endpoint with source CED " + SOURCE_CED + " and time at " + TIME_VARIABLE + ". One moment please...")

url = TIME_API + API_ENDPOINT
payload = {
    'uri': SOURCE_CED,
    'time' : TIME_VARIABLE
}
r = requests.get(url, params=payload)
jsondata = r.json()
print(json.dumps(jsondata, indent=4))
SOURCE_CED_URI=jsondata['uri']
source_geojson_data = get_geom(SOURCE_CED_URI)
g= shape(source_geojson_data)

x_coord = g.centroid.coords[0][1]
y_coord = g.centroid.coords[0][0]

map1_1 = ipy.Map(center=[x_coord, y_coord], zoom=11)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

geo_json1 = GeoJSON(data=source_geojson_data, style = {'color': 'blue', 'opacity':1, 'weight':1.9, 'fillOpacity':0.5})
map1_1.add_layer(geo_json1)
map1_1

Issuing query to /temporal/feature API endpoint with source CED https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/maribyrnong and time at 2021-05-06. One moment please...
{
    "uri": "https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/maribyrnong/2018-07",
    "feature_type": "https://linked.data.gov.au/def/asgs#CommonwealthElectoralDivision",
    "valid_from": "2018-07-01",
    "valid_to": null,
    "code": "230",
    "name": "Maribyrnong",
    "ced_code16": "230",
    "ced_name16": "Maribyrnong",
    "areasqkm16": "63.1908"
}


Map(center=[-37.75516062914489, 144.88991893659147], controls=(ZoomControl(options=['position', 'zoom_in_text'…

## Use case 1.A. Features intersecting at the same time (Fix at a point in time for source feature)

For entity A, in data set X, return identifiers for entities of class 2 from data set Y, that intersect A at time α

Example: For a starting feature of CED Commonwealth Electoral District (CED) of Maribyrnong, return identifiers for the target features that intersected the source CED, on 7 September 2017.

In [10]:
SOURCE_CED = "https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/maribyrnong"
TIME_VARIABLE = "2017-09-07"

In [11]:

datetime_object = datetime.strptime('07/09/2017', '%d/%m/%Y')


wdate_target_ft = widgets.Dropdown(
    options=[("ASGS LGAs", 'https://linked.data.gov.au/def/asgs#LocalGovernmentArea'),
             ("ASGS SEDs", 'https://linked.data.gov.au/def/asgs#StateElectoralDivision'),
             ("ASGS SA2s", 'https://linked.data.gov.au/def/asgs#StatisticalAreaLevel2')
              ],
    description='Feature type:',
    value="https://linked.data.gov.au/def/asgs#LocalGovernmentArea"
)

wdate_date = widgets.DatePicker(
    description='Source Feature Date', 
    value=datetime_object,
    disabled=False
)

In [12]:
display(wdate_target_ft)
display(wdate_date)

Dropdown(description='Feature type:', options=(('ASGS LGAs', 'https://linked.data.gov.au/def/asgs#LocalGovernm…

DatePicker(value=datetime.datetime(2017, 9, 7, 0, 0), description='Source Feature Date')

In [13]:
TO_FEATURE_TYPE = wdate_target_ft.value
#print(wdate_date.value)
TIME_VARIABLE=wdate_date.value

API_ENDPOINT = "/temporal/intersects"

print("Issuing query to "+  API_ENDPOINT + " API endpoint with source CED " + SOURCE_CED + " and time at " + str(TIME_VARIABLE) + " and target features of " + TO_FEATURE_TYPE + ". One moment please...")


url = TIME_API + API_ENDPOINT
payload = {
    'uri': SOURCE_CED,
    'time' : TIME_VARIABLE,
    'toFeatureType' : TO_FEATURE_TYPE
}
r = requests.get(url, params=payload)
jsondata = r.json()
mylist = [[k] for k in jsondata]
df = pd.DataFrame (mylist, columns = ['uri'])
df

Issuing query to /temporal/intersects API endpoint with source CED https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/maribyrnong and time at 2017-09-07 00:00:00 and target features of https://linked.data.gov.au/def/asgs#LocalGovernmentArea. One moment please...


uri


Show the results on a map...

In [14]:
map2 = ipy.Map(center=[x_coord, y_coord], zoom=11)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

source_geojson = GeoJSON(data=source_geojson_data, style = {'color': 'red', 'opacity':1, 'weight':1.9, 'fillOpacity':0.1})
map2.add_layer(source_geojson)

with tqdm(total=len(jsondata)) as pbar:
    for uri in jsondata:
        #print(uri)
        geojson_data = get_geom(uri)
        geo_json1 = GeoJSON(data=geojson_data, style = {
            'stroke': True, 'color': 'black', 'opacity': 0.3, 'weight':1, 
            'fillColor': 'blue', 'fillOpacity':0.1})
        map2.add_layer(geo_json1)
        pbar.update(1)
map2

  0%|          | 0/4 [00:00<?, ?it/s]

Map(center=[-37.75516062914489, 144.88991893659147], controls=(ZoomControl(options=['position', 'zoom_in_text'…

## Use case 1.B. Features intersecting at the same time (Fix at a point in time for both)

For entity A, in data set X, return identifiers for entities of class 2 from data set Y at time b, that intersect A at time α, where time α and b are the same

Example: For a starting feature of CED Commonwealth Electoral District (CED) of Maribyrnong, return identifiers for the target features, LGA regions, that existed at 2017-09-07, that intersected the source CED, on 7 September 2017.

In [15]:
SOURCE_CED = "https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/maribyrnong"
TO_FEATURE_TYPE = "https://linked.data.gov.au/def/asgs#LocalGovernmentArea"
TIME_SOURCE_FEATURE = "2017-09-07"
TIME_TARGET_FEATURE_TYPE = "2017-09-07"

In [16]:

datetime_object = datetime.strptime('07/09/2017', '%d/%m/%Y')


wdate_target_ft = widgets.Dropdown(
    options=[("ASGS LGAs", 'https://linked.data.gov.au/def/asgs#LocalGovernmentArea'),
             ("ASGS SEDs", 'https://linked.data.gov.au/def/asgs#StateElectoralDivision'),
             ("ASGS SA3s", 'https://linked.data.gov.au/def/asgs#StatisticalAreaLevel3'),
             ("ASGS SA2s", 'https://linked.data.gov.au/def/asgs#StatisticalAreaLevel2')
              ],
    description='Feature type:',
    value="https://linked.data.gov.au/def/asgs#LocalGovernmentArea"
)

wdate_date = widgets.DatePicker(
    description='Source Feature Date', 
    value=datetime_object,
    disabled=False
)

wdate_date2 = widgets.DatePicker(
    description='Target FeatureType Date', 
    value=datetime_object,
    disabled=False
)

In [17]:
display(wdate_target_ft)
display(wdate_date)
display(wdate_date2)

Dropdown(description='Feature type:', options=(('ASGS LGAs', 'https://linked.data.gov.au/def/asgs#LocalGovernm…

DatePicker(value=datetime.datetime(2017, 9, 7, 0, 0), description='Source Feature Date')

DatePicker(value=datetime.datetime(2017, 9, 7, 0, 0), description='Target FeatureType Date')

In [18]:
TO_FEATURE_TYPE = wdate_target_ft.value
#print(wdate_date.value)
TIME_VARIABLE=wdate_date.value
TIME_VARIABLE2=wdate_date2.value

API_ENDPOINT = "/temporal/intersects2"

print("Issuing query to "+  API_ENDPOINT + " API endpoint with source CED " + SOURCE_CED + " and time at " + str(TIME_VARIABLE) + " and target features of " + TO_FEATURE_TYPE + " at time " + " and " + str(TIME_VARIABLE2) + ". One moment please...")


url = TIME_API + API_ENDPOINT
payload = {
    'uri': SOURCE_CED,
    'fromTime' : TIME_SOURCE_FEATURE,
    'toTime' : TIME_TARGET_FEATURE_TYPE,
    'toFeatureType' : TO_FEATURE_TYPE
}
r = requests.get(url, params=payload)
print(r.url)
jsondata = r.json()
mylist = [[k] for k in jsondata]
df = pd.DataFrame (mylist, columns = ['uri'])
df

Issuing query to /temporal/intersects2 API endpoint with source CED https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/maribyrnong and time at 2017-09-07 00:00:00 and target features of https://linked.data.gov.au/def/asgs#LocalGovernmentArea at time  and 2017-09-07 00:00:00. One moment please...
http://time-api.loci.cat/api/v1/temporal/intersects2?uri=https%3A%2F%2Flinked.data.gov.au%2Fdataset%2Fasgs%2Fcommonwealthelectoraldivision%2Fmaribyrnong&fromTime=2017-09-07&toTime=2017-09-07&toFeatureType=https%3A%2F%2Flinked.data.gov.au%2Fdef%2Fasgs%23LocalGovernmentArea


uri


In [19]:
map3 = ipy.Map(center=[x_coord, y_coord], zoom=10)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

source_geojson = GeoJSON(data=source_geojson_data, style = {
              'color': 'red', 'opacity':1, 'weight':1.9, 'fillOpacity':0.1})
map3.add_layer(source_geojson)

with tqdm(total=len(jsondata)) as pbar:
    for uri in jsondata:
        #print(uri)
        geojson_data = get_geom(uri)
        geo_json1 = GeoJSON(data=geojson_data, style = {
            'stroke': True, 'color': 'black', 'opacity': 0.3, 'weight':1, 
            'fillColor': 'green', 'fillOpacity':0.2})
        map3.add_layer(geo_json1)
        pbar.update(1)
map3

  0%|          | 0/4 [00:00<?, ?it/s]

Map(center=[-37.75516062914489, 144.88991893659147], controls=(ZoomControl(options=['position', 'zoom_in_text'…

## Use case 1.B. Features intersecting at different times (Specify time points for source feature and target feature types)

For entity A, in data set X, return identifiers for entities of class 2 from data set Y at time b, that intersect A at time α, where time α and b are different

### Change the CED date from 2017-09-07 to 2020-01-01 and keep LGA time the same

First get the 2020-01-01 CED feature

In [20]:
wdate_target_ft = widgets.Dropdown(
    options=[("ASGS LGAs", 'https://linked.data.gov.au/def/asgs#LocalGovernmentArea'),
             ("ASGS CEDs", 'https://linked.data.gov.au/def/asgs#CommonwealthElectoralDivision'),
             ("ASGS SEDs", 'https://linked.data.gov.au/def/asgs#StateElectoralDivision'),
             ("ASGS SA3s", 'https://linked.data.gov.au/def/asgs#StatisticalAreaLevel3'),
             ("ASGS SA2s", 'https://linked.data.gov.au/def/asgs#StatisticalAreaLevel2')
              ],
    description='Feature type:',
    value="https://linked.data.gov.au/def/asgs#LocalGovernmentArea"
)

wdate_source_feature = widgets.Dropdown(
    options=[('2017-01-01', "2017-01-01"), ('2018-01-01', "2018-01-01"),('2019-01-01', "2019-01-01"), ('2020-01-01', "2020-01-01")],
    description='Source Feature Date:',
    value="2020-01-01"
)

wdate_target_features = widgets.Dropdown(
    options=[('2017-01-01', "2017-01-01"), ('2018-01-01', "2018-01-01"),('2019-01-01', "2019-01-01"), ('2020-01-01', "2020-01-01")],
    description='Target Features Date:',
)

In [21]:
display(wdate_target_ft)
display(wdate_source_feature)
display(wdate_target_features)

Dropdown(description='Feature type:', options=(('ASGS LGAs', 'https://linked.data.gov.au/def/asgs#LocalGovernm…

Dropdown(description='Source Feature Date:', index=3, options=(('2017-01-01', '2017-01-01'), ('2018-01-01', '2…

Dropdown(description='Target Features Date:', options=(('2017-01-01', '2017-01-01'), ('2018-01-01', '2018-01-0…

In [22]:
SOURCE_FEATURE = "https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/fraser"
TIME_VARIABLE = wdate_source_feature.value
API_ENDPOINT = "/temporal/feature"

url = TIME_API + API_ENDPOINT
payload = {
    'uri': SOURCE_FEATURE,
    'time' : TIME_VARIABLE
}
r = requests.get(url, params=payload)
jsondata = r.json()
jsondata

{'uri': 'https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/fraser/2018-07',
 'feature_type': 'https://linked.data.gov.au/def/asgs#CommonwealthElectoralDivision',
 'valid_from': '2018-07-01',
 'valid_to': None,
 'code': '214',
 'name': 'Fraser',
 'ced_code16': '214',
 'ced_name16': 'Fraser',
 'areasqkm16': '100.8444'}

In [23]:
SOURCE_FEATURE = jsondata['uri']
#TO_FEATURE_TYPE = "https://linked.data.gov.au/def/asgs#StatisticalAreaLevel1"
#TO_FEATURE_TYPE = "https://linked.data.gov.au/def/asgs#LocalGovernmentArea"
TO_FEATURE_TYPE = wdate_target_ft.value
TIME_SOURCE_FEATURE = wdate_source_feature.value
TIME_TARGET_FEATURE_TYPE = wdate_target_features.value

In [24]:
API_ENDPOINT = "/temporal/intersects2"

url = TIME_API + API_ENDPOINT
payload = {
    'uri': SOURCE_FEATURE,
    'fromTime' : TIME_SOURCE_FEATURE,
    'toTime' : TIME_TARGET_FEATURE_TYPE,
    'toFeatureType' : TO_FEATURE_TYPE
}

source_geojson_data = get_geom(SOURCE_FEATURE)

r = requests.get(url, params=payload)
print(r.url)
jsondata2 = r.json()
mylist = [[k] for k in jsondata2]
df = pd.DataFrame (mylist, columns = ['uri'])
df

http://time-api.loci.cat/api/v1/temporal/intersects2?uri=https%3A%2F%2Flinked.data.gov.au%2Fdataset%2Fasgs%2Fcommonwealthelectoraldivision%2Ffraser%2F2018-07&fromTime=2020-01-01&toTime=2017-01-01&toFeatureType=https%3A%2F%2Flinked.data.gov.au%2Fdef%2Fasgs%23LocalGovernmentArea


uri


In [25]:
source_geojson_data = get_geom(SOURCE_FEATURE)
g= shape(source_geojson_data)
x_coord = g.centroid.coords[0][1]
y_coord = g.centroid.coords[0][0]

map4 = ipy.Map(center=[x_coord, y_coord], zoom=10)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

source_geojson = GeoJSON(data=source_geojson_data, style = {
              'color': 'red', 'opacity':1, 'weight':1.9, 'fillOpacity':0.5})
map4.add_layer(source_geojson)

with tqdm(total=len(jsondata2)) as pbar:
    for uri in jsondata2:
        #print(uri)
        geojson_data = get_geom(uri)
        geo_json1 = GeoJSON(data=geojson_data, style = {
            'stroke': True, 'color': 'black', 'opacity': 1, 'weight':1, 
            'fillColor': 'green', 'fillOpacity':0.2})
        map4.add_layer(geo_json1)
        pbar.update(1)
map4

  0%|          | 0/7 [00:00<?, ?it/s]

Map(center=[-37.7467757320652, 144.8069843868863], controls=(ZoomControl(options=['position', 'zoom_in_text', …

## Use case 2.A. Features intersecting input feature between time A and B.

For entity A, in data set X, return identifiers for entities of class 2 from data set Y, that intersected entity A between times α and β

Example: For a starting feature of CED Commonwealth Electoral District (CED) of Maribyrnong, return identifiers for the target features, ASGS Non-ABS LGAs, that intersected the source CED, on 7 September 2017 and 7 September 2020

In [26]:
#get the fraser feature
SOURCE_CED = "https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/maribyrnong"
TIME_VARIABLE = "2017-09-07"
API_ENDPOINT = "/temporal/feature"

url = TIME_API + API_ENDPOINT
payload = {
    'uri': SOURCE_CED,
    'time' : TIME_VARIABLE
}
r = requests.get(url, params=payload)
print(r.url)
jsondata = r.json()
jsondata

http://time-api.loci.cat/api/v1/temporal/feature?uri=https%3A%2F%2Flinked.data.gov.au%2Fdataset%2Fasgs%2Fcommonwealthelectoraldivision%2Fmaribyrnong&time=2017-09-07


{'uri': 'https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/maribyrnong/2017-07',
 'feature_type': 'https://linked.data.gov.au/def/asgs#CommonwealthElectoralDivision',
 'valid_from': '2017-07-01',
 'valid_to': '2018-06-30',
 'code': '228',
 'name': 'Maribyrnong',
 'ced_code16': '228',
 'ced_name16': 'Maribyrnong',
 'areasqkm16': '72.514600000000002'}

In [27]:
FROM_FEATURE = jsondata['uri']
TO_FEATURE_TYPE = "https://linked.data.gov.au/def/asgs#LocalGovernmentArea"
TIMESPAN_START = "2017-09-07"
TIMESPAN_END = "2020-09-07"

In [28]:
API_ENDPOINT = "/temporal/intersects3"

url = TIME_API + API_ENDPOINT
payload = {
    'uri': FROM_FEATURE,
    'fromTime' : TIMESPAN_START,
    'toTime' : TIMESPAN_END,
    'toFeatureType' : TO_FEATURE_TYPE
}
r = requests.get(url, params=payload)
#print(r.url)
jsondata = r.json()
mydict = {k: sorted(v) for k, v in jsondata.items()}
df = pd.DataFrame (mydict, columns = mydict.keys())
df

https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/maribyrnong/2017-07


In [29]:
from_feature_geom = get_geom(FROM_FEATURE)
g= shape(from_feature_geom)
x_coord_2020 = g.centroid.coords[0][1]
y_coord_2020 = g.centroid.coords[0][0]

map5 = ipy.Map(center=[x_coord_2020, y_coord_2020], zoom=10)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

source_geojson = GeoJSON(data=from_feature_geom, style = {
              'color': 'red', 'opacity':1, 'weight':1.9, 'fillOpacity':0.1})
map5.add_layer(source_geojson)

count = 0
for k in jsondata:
    count = count + len(jsondata[k])

with tqdm(total=count) as pbar:
    for key in jsondata:
        for uri in jsondata[key]:
            #print(uri)
            geojson_data = get_geom(uri)
            geo_json1 = GeoJSON(data=geojson_data, style = {
                'stroke': True, 'color': 'black', 'opacity': 0.3, 'weight':1, 
                'fillColor': 'green', 'fillOpacity':0.2})
            map5.add_layer(geo_json1)
            pbar.update(1)
map5

  0%|          | 0/4 [00:00<?, ?it/s]

Map(center=[-37.7533330724475, 144.86146427671474], controls=(ZoomControl(options=['position', 'zoom_in_text',…

### Now do the reverse

Example: For a starting feature of LGA 27260, return identifiers for the target features, ASGS Non-ABS CEDs, that intersected the source LGA, on 7 September 2017 and 7 September 2020 (there was an election in 2019, so CEDs were changed in 2018).

In [30]:
LGA_URI = "https://linked.data.gov.au/dataset/asgs/localgovernmentarea/25060"
TIME_VARIABLE = "2020-01-01"
API_ENDPOINT = "/temporal/feature"

url = TIME_API + API_ENDPOINT
payload = {
    'uri': LGA_URI,
    'time' : TIME_VARIABLE
}
r = requests.get(url, params=payload)
jsondata = r.json()
jsondata

{'uri': 'https://linked.data.gov.au/dataset/asgs/localgovernmentarea/25060/2019-07',
 'feature_type': 'https://linked.data.gov.au/def/asgs#LocalGovernmentArea',
 'valid_from': '2019-07-01',
 'valid_to': '2020-05-31',
 'code': '25060',
 'name': 'Moonee Valley (C)',
 'lga_code16': '25060',
 'lga_name16': 'Moonee Valley (C)',
 'ste_code16': '2',
 'ste_name16': 'Victoria',
 'areasqkm16': '43.1386'}

In [31]:
FROM_FEATURE = jsondata['uri']
TO_FEATURE_TYPE = "https://linked.data.gov.au/def/asgs#CommonwealthElectoralDivision"
TIMESPAN_START = "2017-09-07"
TIMESPAN_END = "2020-09-07"

In [32]:
API_ENDPOINT = "/temporal/intersects3"

url = TIME_API + API_ENDPOINT
payload = {
    'uri': FROM_FEATURE,
    'fromTime' : TIMESPAN_START,
    'toTime' : TIMESPAN_END,
    'toFeatureType' : TO_FEATURE_TYPE
}
r = requests.get(url, params=payload)
#print(r.url)
jsondata = r.json()
mydict = {k: sorted(v) for k, v in jsondata.items()}
df = pd.DataFrame (mydict, columns = mydict.keys())
df

https://linked.data.gov.au/dataset/asgs/localgovernmentarea/25060/2019-07


In [33]:
jsondata

{'https://linked.data.gov.au/dataset/asgs/localgovernmentarea/25060/2019-07': ['https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/calwell/2018-07',
  'https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/fraser/2018-07',
  'https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/maribyrnong/2018-07',
  'https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/melbourne/2018-07',
  'https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/wills/2018-07',
  'https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/calwell',
  'https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/maribyrnong',
  'https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/melbourne',
  'https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/wills',
  'https://linked.data.gov.au/dataset/asgs/commonwealthelectoraldivision/fraser']}

In [34]:
from_feature_geom = get_geom(FROM_FEATURE)
g= shape(from_feature_geom)
x_coord_2020 = g.centroid.coords[0][1]
y_coord_2020 = g.centroid.coords[0][0]

map6 = ipy.Map(center=[x_coord_2020, y_coord_2020], zoom=10)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

source_geojson = GeoJSON(data=from_feature_geom, style = {
              'color': 'red', 'opacity':1, 'weight':1.9, 'fillOpacity':0.1})
map6.add_layer(source_geojson)

count = 0
for k in jsondata:
    count = count + len(jsondata[k])

with tqdm(total=count) as pbar:
    for key in jsondata:
        for uri in jsondata[key]:
            #print(uri)
            geojson_data = get_geom(uri)
            geo_json1 = GeoJSON(data=geojson_data, style = {
                'stroke': True, 'color': 'black', 'opacity': 0.3, 'weight':1, 
                'fillColor': 'green', 'fillOpacity':0.2})
            map6.add_layer(geo_json1)
            pbar.update(1)
map6

  0%|          | 0/10 [00:00<?, ?it/s]

Map(center=[-37.749499924536416, 144.89576166688516], controls=(ZoomControl(options=['position', 'zoom_in_text…